In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('wines')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased       
            wait(driver, 400).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            if current_rows_number % 200 == 0:
              print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [4]:
from wine import Wine

argentina_wine_url = "https://www.vivino.com/explore?e=eJwNyzEKgDAMRuHbZK6D4795A3ESkVhjKdhWklL09nZ5y8dLioFSzHCU-MXoHPkPy0y-Z6Kna7jQWKNUvqkcUK4xB9u5iXIQKjjFfL9s3cBKBs8_v4QdNg=="
argentina_wine = Wine(argentina_wine_url)
argentina_wine.setup_driver()
argentina_wine.run_timestamp()
lazyloading(argentina_wine.driver)
argentina_wine.run_timestamp()

Current Timestamp:  30-Jan-2020 (17:50:34.056950)
Total Number:  37676
Curr Row Number:  200
Curr Row Number:  400
Curr Row Number:  600
Curr Row Number:  800
Curr Row Number:  1000
Curr Row Number:  1200
Curr Row Number:  1400
Curr Row Number:  1600
Current Timestamp:  30-Jan-2020 (18:00:10.730463)


In [5]:
argentina_html = BeautifulSoup(argentina_wine.driver.page_source, 'lxml')
argentina_div = argentina_html.find("div", {"class": "explorerPage__results--3wqLw"})
argentina_rows = argentina_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [6]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "country": None, "continent": None, "ratings": None, "num_ratings": None, "reviews": None, "image_url": None
}

for row in argentina_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    new_row['country'] = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['continent'] = 'South America'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['review'] = review_div.text
    image_div = row.find("div", {"class": "cleanWineCard__bottleShotWrapper--nymTj"})
    if image_div:
        new_row['image_url'] = image_div.find("div").find_next('img')['src'].strip("//")
    all_rows.append(new_row)

In [7]:
all_rows[0]

{'title': 'Cobos Malbec 2012',
 'country': 'Argentina',
 'continent': 'South America',
 'ratings': '4.8',
 'num_ratings': 857,
 'reviews': None,
 'image_url': 'images.vivino.com/thumbs/D1Mf1fYnRnageFjtfLXdFg_pb_x300.png'}